In [ ]:
import torch
import torch.nn as nn
device = "cuda" if torch.cuda.is_available() else "cpu"
from torch.utils.data import Dataset, DataLoader 
from torchvision import datasets, transforms
import matplotlib.pyplot as plt 
import numpy as np

In [ ]:
transform = transforms.ToTensor()

data_path = '../data_cifar/'
cifar10_train = datasets.CIFAR10(data_path, train=True, download=True, transform=transform) 
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=transform)
print("Training: ", len(cifar10_train)) #Training: 50000 
print("Validating: ", len(cifar10_val)) #Testing: 10000

In [ ]:
class CIFAR10Dataset(Dataset):
    def __init__(self, x, y):
        x = torch.tensor(x, dtype=torch.float) / 255      # scalling
        x = x.reshape(-1, 3, 32, 32)
        y = torch.tensor(y)
        self.x, self.y = x, y 
    def __getitem__(self, ix):
        x, y = self.x[ix], self.y[ix]        
        return x.to(device), y.to(device)
    def __len__(self): 
        return len(self.x)

In [ ]:
train_dataset = CIFAR10Dataset(cifar10_train.data, cifar10_train.targets)
valid_dataset = CIFAR10Dataset(cifar10_val.data, cifar10_val.targets)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True) 
val_loader = DataLoader(valid_dataset, batch_size=10000, shuffle=False)

In [ ]:
from torch.optim import Adam

def get_baseline_model():
    model = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Conv2d(64, 128, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(4608, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
    ).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=1e-3)
    return model, loss_fn, optimizer

In [ ]:
def train_batch(x, y, model, optimizer, loss_fn):
    model.train()
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

In [ ]:
@torch.no_grad()
def val_loss(x, y, model, loss_fn):
    model.eval()
    prediction = model(x)
    val_loss = loss_fn(prediction, y)
    return val_loss.item()

In [ ]:
from torchsummary import summary

baseline_model, baseline_loss_fn, baseline_optimizer = get_baseline_model()
summary(baseline_model, input_size=(3, 32, 32))
# summary가 없는 사람은 감점

In [ ]:
from tqdm.auto import tqdm
epochs = 20

def train(model, loss_fn, optimizer, epochs):
    train_losses, train_accuracies = [], []
    val_losses, val_accuracies = [], []

    for epoch in tqdm(range(epochs)):
        train_epoch_losses, train_epoch_accuracies = [], []
        for ix, batch in enumerate(iter(train_loader)):
            x, y = batch
            batch_loss = train_batch(x, y, model, optimizer, loss_fn)
            train_epoch_losses.append(batch_loss)        
        train_epoch_loss = np.array(train_epoch_losses).mean()

        for ix, batch in enumerate(iter(train_loader)):
            x, y = batch
            is_correct = accuracy(x, y, model)
            train_epoch_accuracies.extend(is_correct)
        train_epoch_accuracy = np.mean(train_epoch_accuracies)

        for ix, batch in enumerate(iter(val_loader)):
            x, y = batch
            val_is_correct = accuracy(x, y, model)
            validation_loss = val_loss(x, y, model, loss_fn)
        val_epoch_accuracy = np.mean(val_is_correct)

        train_losses.append(train_epoch_loss)
        train_accuracies.append(train_epoch_accuracy)
        val_losses.append(validation_loss)
        val_accuracies.append(val_epoch_accuracy)
        
    return train_losses, train_accuracies, val_losses, val_accuracies

baseline_train_losses, baseline_train_accuracies, baseline_val_losses, baseline_val_accuracies = train(baseline_model, baseline_loss_fn, baseline_optimizer, epochs)

In [ ]:
def plot_images(train_loss, train_accuracy, val_loss, val_accuracy, epochs):
  epochs = np.arange(epochs) + 1
  plt.figure(figsize=(20, 5))
  plt.subplot(121)
  plt.title('Loss value over increasing epochs')
  plt.plot(epochs, train_loss, label='Training Loss', color='blue', linestyle='--')
  plt.plot(epochs, val_loss, label='Validation Loss', color='red')
  plt.legend()
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.title('Loss value over increasing epochs')
  
  plt.subplot(122)
  plt.title('Accuracy value over increasing epochs')
  plt.plot(epochs, train_accuracy, label='Training Accuracy', color='blue', linestyle='--')
  plt.plot(epochs, val_accuracy, label='Validation Accuracy', color='red')
  plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_yticks()]) 
  plt.legend()
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy(%)')
  plt.title('Accuracy value over increasing epochs')

  plt.tight_layout()
  plt.show()

In [ ]:
plot_images(baseline_train_losses, baseline_train_accuracies, baseline_val_losses, baseline_val_accuracies, epochs)

# 어떠한 조정/hyperparameter 등이 사용되었는지 명확히 기술
- 기술이 없으면 감점